In [38]:
# import geopandas as gpd
# import rasterio
# from matplotlib import pyplot as plt
# import shapely
# from rasterstats import point_query
# from pyproj import Proj, transform, Transformer

# Database

In [1]:
# import mysql.connector
import sqlite3
import pandas as pd
import datetime as dt
import numpy as np 
import requests

In [40]:
# conn = mysql.connector.connect(
#     host='localhost',
#     user='root',
#     password='alvin',
#     database= 'breatheeasy'
# )

# conn = sqlite3.connect('G:/uoa/COMPSCI778internship/778project/data/database/breatheEasy.db')
# cursor = conn.cursor()

## Air quality

### AQI

In [2]:
full_range = pd.date_range(start='2020-01-01', end='2023-08-23', freq='D')
full_range = pd.DataFrame({'Date':full_range})
full_range

,Date
0,2020-01-01
1,2020-01-02
2,2020-01-03
3,2020-01-04
4,2020-01-05
...,...
1326,2023-08-19
1327,2023-08-20
1328,2023-08-21
1329,2023-08-22


In [3]:
heAQI = pd.read_csv('../778project/data/airquality/auckland/AQI/hendersonAQI_h.csv', skiprows=4)
heAQI

,Timestamp (UTC+12:00),Value (AQI)
0,2018-01-01 00:00:00,17
1,2018-01-01 01:00:00,21
2,2018-01-01 02:00:00,23
3,2018-01-01 03:00:00,24
4,2018-01-01 04:00:00,24
...,...,...
48121,2023-12-31 19:00:00,18
48122,2023-12-31 20:00:00,18
48123,2023-12-31 21:00:00,17
48124,2023-12-31 22:00:00,17


In [4]:
heAQI.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (AQI)':'AQI_1'}, inplace=True)
heAQI['Date'] = pd.to_datetime(heAQI['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
heAQI['Date'] = pd.to_datetime(heAQI['Date'])
heAQI = heAQI.groupby('Date')['AQI_1'].mean().reset_index()
heAQI = heAQI[(heAQI['Date']>='2020-01-01') & (heAQI['Date']<='2023-08-23')]
heAQI = full_range.merge(heAQI, on='Date', how='left')
heAQI

,Date,AQI_1
0,2020-01-01,35.000000
1,2020-01-02,18.041667
2,2020-01-03,23.041667
3,2020-01-04,37.500000
4,2020-01-05,42.750000
...,...,...
1326,2023-08-19,24.708333
1327,2023-08-20,13.375000
1328,2023-08-21,21.347826
1329,2023-08-22,30.875000


In [5]:
aqi_df = full_range.merge(heAQI, on='Date', how='left')
aqi_df

,Date,AQI_1
0,2020-01-01,35.000000
1,2020-01-02,18.041667
2,2020-01-03,23.041667
3,2020-01-04,37.500000
4,2020-01-05,42.750000
...,...,...
1326,2023-08-19,24.708333
1327,2023-08-20,13.375000
1328,2023-08-21,21.347826
1329,2023-08-22,30.875000


In [6]:
peAQI = pd.read_csv('../778project/data/airquality/auckland/AQI/penroseAQI_h.csv', skiprows=4)
peAQI

,Timestamp (UTC+12:00),Value (AQI)
0,2018-01-01 00:00:00,27
1,2018-01-01 01:00:00,26
2,2018-01-01 02:00:00,25
3,2018-01-01 03:00:00,24
4,2018-01-01 04:00:00,24
...,...,...
47501,2023-12-31 19:00:00,22
47502,2023-12-31 20:00:00,20
47503,2023-12-31 21:00:00,19
47504,2023-12-31 22:00:00,19


In [7]:
peAQI.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (AQI)':'AQI_2'}, inplace=True)
peAQI['Date'] = pd.to_datetime(peAQI['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
peAQI['Date'] = pd.to_datetime(peAQI['Date'])
peAQI = peAQI.groupby('Date')['AQI_2'].mean().reset_index()
peAQI = peAQI[(peAQI['Date']>='2020-01-01') & (peAQI['Date']<='2023-08-23')]
peAQI = full_range.merge(peAQI, on='Date', how='left')
peAQI

,Date,AQI_2
0,2020-01-01,35.041667
1,2020-01-02,20.166667
2,2020-01-03,28.625000
3,2020-01-04,48.666667
4,2020-01-05,51.833333
...,...,...
1326,2023-08-19,39.416667
1327,2023-08-20,18.666667
1328,2023-08-21,23.708333
1329,2023-08-22,42.086957


In [8]:
aqi_df = aqi_df.merge(peAQI, on='Date', how='left')
aqi_df

,Date,AQI_1,AQI_2
0,2020-01-01,35.000000,35.041667
1,2020-01-02,18.041667,20.166667
2,2020-01-03,23.041667,28.625000
3,2020-01-04,37.500000,48.666667
4,2020-01-05,42.750000,51.833333
...,...,...,...
1326,2023-08-19,24.708333,39.416667
1327,2023-08-20,13.375000,18.666667
1328,2023-08-21,21.347826,23.708333
1329,2023-08-22,30.875000,42.086957


In [10]:
quAQI = pd.read_csv('../778project/data/airquality/auckland/AQI/queenstreetAQI_h.csv', skiprows=4)
quAQI

,Timestamp (UTC+12:00),Value (AQI)
0,2018-01-01 00:00:00,24
1,2018-01-01 01:00:00,24
2,2018-01-01 02:00:00,25
3,2018-01-01 03:00:00,25
4,2018-01-01 04:00:00,25
...,...,...
44612,2023-08-23 00:00:00,56
44613,2023-08-23 01:00:00,56
44614,2023-08-23 02:00:00,55
44615,2023-08-23 03:00:00,55


In [11]:
quAQI.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (AQI)':'AQI_3'}, inplace=True)
quAQI['Date'] = pd.to_datetime(quAQI['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
quAQI['Date'] = pd.to_datetime(quAQI['Date'])
quAQI = quAQI.groupby('Date')['AQI_3'].mean().reset_index()
quAQI = quAQI[(quAQI['Date']>='2020-01-01') & (quAQI['Date']<='2023-08-23')]
quAQI = full_range.merge(quAQI, on='Date', how='left')
quAQI

,Date,AQI_3
0,2020-01-01,37.250000
1,2020-01-02,23.500000
2,2020-01-03,31.000000
3,2020-01-04,46.541667
4,2020-01-05,47.541667
...,...,...
1326,2023-08-19,45.333333
1327,2023-08-20,31.708333
1328,2023-08-21,41.166667
1329,2023-08-22,57.347826


In [12]:
aqi_df = aqi_df.merge(quAQI, on='Date', how='left')
aqi_df

,Date,AQI_1,AQI_2,AQI_3
0,2020-01-01,35.000000,35.041667,37.250000
1,2020-01-02,18.041667,20.166667,23.500000
2,2020-01-03,23.041667,28.625000,31.000000
3,2020-01-04,37.500000,48.666667,46.541667
4,2020-01-05,42.750000,51.833333,47.541667
...,...,...,...,...
1326,2023-08-19,24.708333,39.416667,45.333333
1327,2023-08-20,13.375000,18.666667,31.708333
1328,2023-08-21,21.347826,23.708333,41.166667
1329,2023-08-22,30.875000,42.086957,57.347826


In [14]:
taAQI = pd.read_csv('../778project/data/airquality/auckland/AQI/takapunaAQI_h.csv', skiprows=4)
taAQI

,Timestamp (UTC+12:00),Value (AQI)
0,2018-01-01 00:00:00,18
1,2018-01-01 01:00:00,20
2,2018-01-01 02:00:00,20
3,2018-01-01 03:00:00,20
4,2018-01-01 04:00:00,20
...,...,...
47173,2023-12-31 19:00:00,23
47174,2023-12-31 20:00:00,22
47175,2023-12-31 21:00:00,21
47176,2023-12-31 22:00:00,21


In [15]:
taAQI.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (AQI)':'AQI_4'}, inplace=True)
taAQI['Date'] = pd.to_datetime(taAQI['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
taAQI['Date'] = pd.to_datetime(taAQI['Date'])
taAQI = taAQI.groupby('Date')['AQI_4'].mean().reset_index()
taAQI = taAQI[(taAQI['Date']>='2020-01-01') & (taAQI['Date']<='2023-08-23')]
taAQI = full_range.merge(taAQI, on='Date', how='left')
taAQI

,Date,AQI_4
0,2020-01-01,37.208333
1,2020-01-02,21.875000
2,2020-01-03,29.041667
3,2020-01-04,40.666667
4,2020-01-05,39.416667
...,...,...
1326,2023-08-19,35.625000
1327,2023-08-20,19.333333
1328,2023-08-21,27.521739
1329,2023-08-22,34.375000


In [16]:
aqi_df = aqi_df.merge(taAQI, on='Date', how='left')
aqi_df

,Date,AQI_1,AQI_2,AQI_3,AQI_4
0,2020-01-01,35.000000,35.041667,37.250000,37.208333
1,2020-01-02,18.041667,20.166667,23.500000,21.875000
2,2020-01-03,23.041667,28.625000,31.000000,29.041667
3,2020-01-04,37.500000,48.666667,46.541667,40.666667
4,2020-01-05,42.750000,51.833333,47.541667,39.416667
...,...,...,...,...,...
1326,2023-08-19,24.708333,39.416667,45.333333,35.625000
1327,2023-08-20,13.375000,18.666667,31.708333,19.333333
1328,2023-08-21,21.347826,23.708333,41.166667,27.521739
1329,2023-08-22,30.875000,42.086957,57.347826,34.375000


In [17]:
aqi_df['Daily Average AQI'] = aqi_df[['AQI_1', 'AQI_2', 'AQI_3', 'AQI_4']].mean(axis=1)
aqi_df = aqi_df[['Date', 'Daily Average AQI']]
aqi_df.head(10)

,Date,Daily Average AQI
0,2020-01-01,36.125000
1,2020-01-02,20.895833
2,2020-01-03,27.927083
3,2020-01-04,43.343750
4,2020-01-05,45.385417
5,2020-01-06,44.343750
6,2020-01-07,52.666667
7,2020-01-08,35.968750
8,2020-01-09,30.427083
9,2020-01-10,32.333333


### NOx

In [18]:
peNO = pd.read_csv('../778project/data/airquality/auckland/NOx/penroseNOx_h.csv', skiprows=4)
peNO

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,16.4
1,2020-01-01 01:00:00,23.7
2,2020-01-01 02:00:00,16.2
3,2020-01-01 03:00:00,16.7
4,2020-01-01 04:00:00,18.5
...,...,...
33927,2023-12-31 20:00:00,2.7
33928,2023-12-31 21:00:00,2.9
33929,2023-12-31 22:00:00,3.8
33930,2023-12-31 23:00:00,5.8


In [19]:
peNO.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'NO_1'}, inplace=True)
peNO['Date'] = pd.to_datetime(peNO['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
peNO['Date'] = pd.to_datetime(peNO['Date'])
peNO = peNO.groupby('Date')['NO_1'].mean().reset_index()
peNO = peNO[(peNO['Date']>='2020-01-01') & (peNO['Date']<='2023-08-23')]
peNO = full_range.merge(peNO, on='Date', how='left')
peNO

,Date,NO_1
0,2020-01-01,9.841667
1,2020-01-02,7.334783
2,2020-01-03,10.704167
3,2020-01-04,8.086957
4,2020-01-05,6.643478
...,...,...
1326,2023-08-19,15.225000
1327,2023-08-20,8.430435
1328,2023-08-21,16.825000
1329,2023-08-22,45.526087


In [20]:
no_df = full_range.merge(peNO, on='Date', how='left')
no_df

,Date,NO_1
0,2020-01-01,9.841667
1,2020-01-02,7.334783
2,2020-01-03,10.704167
3,2020-01-04,8.086957
4,2020-01-05,6.643478
...,...,...
1326,2023-08-19,15.225000
1327,2023-08-20,8.430435
1328,2023-08-21,16.825000
1329,2023-08-22,45.526087


In [21]:
quNO = pd.read_csv('../778project/data/airquality/auckland/NOx/queenstreetNOx_h.csv', skiprows=4)
quNO

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,23.3
1,2020-01-01 01:00:00,21.9
2,2020-01-01 02:00:00,45.3
3,2020-01-01 03:00:00,57.2
4,2020-01-01 04:00:00,82.6
...,...,...
31273,2023-08-23 00:00:00,134.4
31274,2023-08-23 01:00:00,86.0
31275,2023-08-23 02:00:00,66.6
31276,2023-08-23 03:00:00,57.2


In [22]:
quNO.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'NO_2'}, inplace=True)
quNO['Date'] = pd.to_datetime(quNO['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
quNO['Date'] = pd.to_datetime(quNO['Date'])
quNO = quNO.groupby('Date')['NO_2'].mean().reset_index()
quNO = quNO[(quNO['Date']>='2020-01-01') & (quNO['Date']<='2023-08-23')]
quNO = full_range.merge(quNO, on='Date', how='left')
quNO

,Date,NO_2
0,2020-01-01,59.604167
1,2020-01-02,48.386957
2,2020-01-03,88.416667
3,2020-01-04,64.100000
4,2020-01-05,115.495652
...,...,...
1326,2023-08-19,44.112500
1327,2023-08-20,105.117391
1328,2023-08-21,69.775000
1329,2023-08-22,90.526087


In [23]:
no_df = no_df.merge(quNO, on='Date', how='left')
no_df

,Date,NO_1,NO_2
0,2020-01-01,9.841667,59.604167
1,2020-01-02,7.334783,48.386957
2,2020-01-03,10.704167,88.416667
3,2020-01-04,8.086957,64.100000
4,2020-01-05,6.643478,115.495652
...,...,...,...
1326,2023-08-19,15.225000,44.112500
1327,2023-08-20,8.430435,105.117391
1328,2023-08-21,16.825000,69.775000
1329,2023-08-22,45.526087,90.526087


In [24]:
taNO = pd.read_csv('../778project/data/airquality/auckland/NOx/takapunaNOx_h.csv', skiprows=4)
taNO

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,20.6
1,2020-01-01 01:00:00,25.9
2,2020-01-01 02:00:00,24.0
3,2020-01-01 03:00:00,18.5
4,2020-01-01 04:00:00,11.9
...,...,...
33436,2023-12-31 20:00:00,8.4
33437,2023-12-31 21:00:00,12.8
33438,2023-12-31 22:00:00,8.2
33439,2023-12-31 23:00:00,4.3


In [25]:
taNO.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'NO_3'}, inplace=True)
taNO['Date'] = pd.to_datetime(taNO['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
taNO['Date'] = pd.to_datetime(taNO['Date'])
taNO = taNO.groupby('Date')['NO_3'].mean().reset_index()
taNO = taNO[(taNO['Date']>='2018-03-25') & (taNO['Date']<='2023-08-23')]
taNO = full_range.merge(taNO, on='Date', how='left')
taNO

,Date,NO_3
0,2020-01-01,12.395833
1,2020-01-02,6.265217
2,2020-01-03,9.020833
3,2020-01-04,5.239130
4,2020-01-05,6.578261
...,...,...
1326,2023-08-19,40.970833
1327,2023-08-20,23.386957
1328,2023-08-21,14.104167
1329,2023-08-22,33.556522


In [26]:
no_df = no_df.merge(taNO, on='Date', how='left')
no_df

,Date,NO_1,NO_2,NO_3
0,2020-01-01,9.841667,59.604167,12.395833
1,2020-01-02,7.334783,48.386957,6.265217
2,2020-01-03,10.704167,88.416667,9.020833
3,2020-01-04,8.086957,64.100000,5.239130
4,2020-01-05,6.643478,115.495652,6.578261
...,...,...,...,...
1326,2023-08-19,15.225000,44.112500,40.970833
1327,2023-08-20,8.430435,105.117391,23.386957
1328,2023-08-21,16.825000,69.775000,14.104167
1329,2023-08-22,45.526087,90.526087,33.556522


In [27]:
heNO = pd.read_csv('../778project/data/airquality/auckland/NOx/hendersonNOx_h.csv', skiprows=4)
heNO

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,17.7
1,2020-01-01 01:00:00,19.4
2,2020-01-01 02:00:00,11.0
3,2020-01-01 03:00:00,12.5
4,2020-01-01 04:00:00,9.1
...,...,...
18465,2023-12-31 20:00:00,1.4
18466,2023-12-31 21:00:00,1.8
18467,2023-12-31 22:00:00,1.4
18468,2023-12-31 23:00:00,1.5


In [28]:
heNO.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'NO_4'}, inplace=True)
heNO['Date'] = pd.to_datetime(heNO['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
heNO['Date'] = pd.to_datetime(heNO['Date'])
heNO = heNO.groupby('Date')['NO_4'].mean().reset_index()
heNO = heNO[(heNO['Date']>='2020-01-01') & (heNO['Date']<='2023-08-23')]
heNO = full_range.merge(heNO, on='Date', how='left')
heNO

,Date,NO_4
0,2020-01-01,4.516667
1,2020-01-02,1.013043
2,2020-01-03,0.437500
3,2020-01-04,0.300000
4,2020-01-05,0.936364
...,...,...
1326,2023-08-19,21.666667
1327,2023-08-20,4.034783
1328,2023-08-21,1.670833
1329,2023-08-22,16.947826


In [29]:
no_df = no_df.merge(heNO, on='Date', how='left')
no_df

,Date,NO_1,NO_2,NO_3,NO_4
0,2020-01-01,9.841667,59.604167,12.395833,4.516667
1,2020-01-02,7.334783,48.386957,6.265217,1.013043
2,2020-01-03,10.704167,88.416667,9.020833,0.437500
3,2020-01-04,8.086957,64.100000,5.239130,0.300000
4,2020-01-05,6.643478,115.495652,6.578261,0.936364
...,...,...,...,...,...
1326,2023-08-19,15.225000,44.112500,40.970833,21.666667
1327,2023-08-20,8.430435,105.117391,23.386957,4.034783
1328,2023-08-21,16.825000,69.775000,14.104167,1.670833
1329,2023-08-22,45.526087,90.526087,33.556522,16.947826


In [30]:
no_df['Daily Average NOx (ug/m3)'] = no_df[['NO_1', 'NO_2', 'NO_3', 'NO_4']].mean(axis=1)
no_df = no_df[['Date', 'Daily Average NOx (ug/m3)']]
no_df.head(10)

,Date,Daily Average NOx (ug/m3)
0,2020-01-01,21.589583
1,2020-01-02,15.750000
2,2020-01-03,27.144792
3,2020-01-04,19.431522
4,2020-01-05,32.413439
5,2020-01-06,26.388043
6,2020-01-07,27.240625
7,2020-01-08,40.054348
8,2020-01-09,36.416667
9,2020-01-10,33.520652


### O3

In [31]:
o3 = pd.read_csv('../778project/data/airquality/auckland/O3/patumahoeO3_h.csv', skiprows=4)
o3

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,24.00
1,2020-01-01 01:00:00,15.00
2,2020-01-01 02:00:00,16.00
3,2020-01-01 03:00:00,13.00
4,2020-01-01 04:00:00,14.00
...,...,...
33380,2023-12-31 20:00:00,49.79
33381,2023-12-31 21:00:00,52.61
33382,2023-12-31 22:00:00,56.46
33383,2023-12-31 23:00:00,48.51


In [32]:
o3.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'Daily Average O3 (ug/m3)'}, inplace=True)
o3['Date'] = pd.to_datetime(o3['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
o3['Date'] = pd.to_datetime(o3['Date'])
o3 = o3.groupby('Date')['Daily Average O3 (ug/m3)'].mean().reset_index()
o3 = o3[(o3['Date']>='2020-01-01') & (o3['Date']<='2023-08-23')]
o3_df = full_range.merge(o3, on='Date', how='left')
o3_df

,Date,Daily Average O3 (ug/m3)
0,2020-01-01,29.608696
1,2020-01-02,37.869565
2,2020-01-03,41.826087
3,2020-01-04,38.608696
4,2020-01-05,40.136364
...,...,...
1326,2023-08-19,48.765833
1327,2023-08-20,52.997917
1328,2023-08-21,62.969583
1329,2023-08-22,50.296250


### SO2

In [33]:
so2 = pd.read_csv('../778project/data/airquality/auckland/SO2/penroseSO2_h.csv', skiprows=4)
so2

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,-0.1
1,2020-01-01 00:00:01,NaN
2,2020-01-01 01:59:59,NaN
3,2020-01-01 02:00:00,-0.1
4,2020-01-01 03:00:00,0.1
...,...,...
34825,2023-12-31 20:00:00,0.6
34826,2023-12-31 21:00:00,2.2
34827,2023-12-31 22:00:00,3.0
34828,2023-12-31 23:00:00,2.6


In [34]:
so2.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'Daily Average SO2 (ug/m3)'}, inplace=True)
so2['Date'] = pd.to_datetime(so2['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
so2['Date'] = pd.to_datetime(so2['Date'])
so2 = so2.groupby('Date')['Daily Average SO2 (ug/m3)'].mean().reset_index()
so2 = so2[(so2['Date']>='2020-01-01') & (so2['Date']<='2023-08-23')]
so2_df = full_range.merge(so2, on='Date', how='left')
so2_df

,Date,Daily Average SO2 (ug/m3)
0,2020-01-01,-0.004348
1,2020-01-02,0.016667
2,2020-01-03,0.004348
3,2020-01-04,0.000000
4,2020-01-05,-0.018182
...,...,...
1326,2023-08-19,0.952174
1327,2023-08-20,1.233333
1328,2023-08-21,1.265217
1329,2023-08-22,1.358333


### PM2.5

In [35]:
pePM25 = pd.read_csv('../778project/data/airquality/auckland/PM2.5/penrosePM2.5_h.csv', skiprows=4)
pePM25

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,11.9
1,2020-01-01 01:00:00,9.3
2,2020-01-01 02:00:00,7.9
3,2020-01-01 03:00:00,7.8
4,2020-01-01 04:00:00,9.4
...,...,...
31984,2023-12-23 01:00:00,5.1
31985,2023-12-23 02:00:00,1.5
31986,2023-12-23 03:00:00,-3.4
31987,2023-12-23 04:00:00,0.7


In [36]:
pePM25.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'pm_1'}, inplace=True)
pePM25['Date'] = pd.to_datetime(pePM25['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
pePM25['Date'] = pd.to_datetime(pePM25['Date'])
pePM25 = pePM25.groupby('Date')['pm_1'].mean().reset_index()
pePM25 = pePM25[(pePM25['Date']>='2020-01-01') & (pePM25['Date']<='2023-08-23')]
pm25_df = full_range.merge(pePM25, on='Date', how='left')
pm25_df

,Date,pm_1
0,2020-01-01,7.087500
1,2020-01-02,5.120833
2,2020-01-03,10.337500
3,2020-01-04,9.812500
4,2020-01-05,8.345833
...,...,...
1326,2023-08-19,6.416667
1327,2023-08-20,1.587500
1328,2023-08-21,3.473684
1329,2023-08-22,5.785000


In [37]:
quPM25 = pd.read_csv('../778project/data/airquality/auckland/PM2.5/queenstreetPM2.5_h.csv', skiprows=4)
quPM25

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,10.6
1,2020-01-01 01:00:00,10.7
2,2020-01-01 02:00:00,9.8
3,2020-01-01 03:00:00,10.3
4,2020-01-01 04:00:00,12.0
...,...,...
29530,2023-08-23 00:00:00,13.2
29531,2023-08-23 01:00:00,12.7
29532,2023-08-23 02:00:00,12.2
29533,2023-08-23 03:00:00,11.0


In [38]:
quPM25.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'pm_2'}, inplace=True)
quPM25['Date'] = pd.to_datetime(quPM25['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
quPM25['Date'] = pd.to_datetime(quPM25['Date'])
quPM25 = quPM25.groupby('Date')['pm_2'].mean().reset_index()
quPM25 = quPM25[(quPM25['Date']>='2020-01-01') & (quPM25['Date']<='2023-08-23')]
pm25_df = pm25_df.merge(quPM25, on='Date', how='left')
pm25_df

,Date,pm_1,pm_2
0,2020-01-01,7.087500,7.762500
1,2020-01-02,5.120833,5.270833
2,2020-01-03,10.337500,9.979167
3,2020-01-04,9.812500,9.662500
4,2020-01-05,8.345833,8.239130
...,...,...,...
1326,2023-08-19,6.416667,10.579167
1327,2023-08-20,1.587500,6.304167
1328,2023-08-21,3.473684,11.483333
1329,2023-08-22,5.785000,11.116667


In [39]:
taPM25 = pd.read_csv('../778project/data/airquality/auckland/PM2.5/takapunaPM2.5_h.csv', skiprows=4)
taPM25

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,13.1
1,2020-01-01 01:00:00,14.5
2,2020-01-01 02:00:00,13.2
3,2020-01-01 03:00:00,11.9
4,2020-01-01 04:00:00,10.6
...,...,...
22129,2023-12-31 20:00:00,2.6
22130,2023-12-31 21:00:00,3.3
22131,2023-12-31 22:00:00,3.0
22132,2023-12-31 23:00:00,4.7


In [40]:
taPM25.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'pm_3'}, inplace=True)
taPM25['Date'] = pd.to_datetime(taPM25['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
taPM25['Date'] = pd.to_datetime(taPM25['Date'])
taPM25 = taPM25.groupby('Date')['pm_3'].mean().reset_index()
taPM25 = taPM25[(taPM25['Date']>='2020-01-01') & (taPM25['Date']<='2023-08-23')]
pm25_df = pm25_df.merge(taPM25, on='Date', how='left')
pm25_df

,Date,pm_1,pm_2,pm_3
0,2020-01-01,7.087500,7.762500,7.850000
1,2020-01-02,5.120833,5.270833,4.970833
2,2020-01-03,10.337500,9.979167,10.104167
3,2020-01-04,9.812500,9.662500,9.283333
4,2020-01-05,8.345833,8.239130,8.216667
...,...,...,...,...
1326,2023-08-19,6.416667,10.579167,7.837500
1327,2023-08-20,1.587500,6.304167,4.208333
1328,2023-08-21,3.473684,11.483333,8.954167
1329,2023-08-22,5.785000,11.116667,8.420833


In [41]:
pm25_df['Daily Average PM2.5 (ug/m3)'] = pm25_df[['pm_1', 'pm_2', 'pm_3']].mean(axis=1)
pm25_df = pm25_df[['Date', 'Daily Average PM2.5 (ug/m3)']]
pm25_df

,Date,Daily Average PM2.5 (ug/m3)
0,2020-01-01,7.566667
1,2020-01-02,5.120833
2,2020-01-03,10.140278
3,2020-01-04,9.586111
4,2020-01-05,8.267210
...,...,...
1326,2023-08-19,8.277778
1327,2023-08-20,4.033333
1328,2023-08-21,7.970395
1329,2023-08-22,8.440833


### PM10

In [42]:
hePM10 = pd.read_csv('../778project/data/airquality/auckland/PM10/hendersonPM10_h.csv', skiprows=4)
hePM10

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,35.0
1,2020-01-01 01:00:00,71.5
2,2020-01-01 02:00:00,29.1
3,2020-01-01 03:00:00,18.9
4,2020-01-01 04:00:00,17.0
...,...,...
18806,2023-12-31 20:00:00,-3.6
18807,2023-12-31 21:00:00,8.6
18808,2023-12-31 22:00:00,4.5
18809,2023-12-31 23:00:00,0.9


In [43]:
hePM10.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'pm_1'}, inplace=True)
hePM10['Date'] = pd.to_datetime(hePM10['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
hePM10['Date'] = pd.to_datetime(hePM10['Date'])
hePM10 = hePM10.groupby('Date')['pm_1'].mean().reset_index()
hePM10 = hePM10[(hePM10['Date']>='2020-01-01') & (hePM10['Date']<='2023-08-23')]
pm10_df = full_range.merge(hePM10, on='Date', how='left')
pm10_df

,Date,pm_1
0,2020-01-01,14.895833
1,2020-01-02,8.429167
2,2020-01-03,16.066667
3,2020-01-04,21.095833
4,2020-01-05,18.772727
...,...,...
1326,2023-08-19,10.154167
1327,2023-08-20,5.558333
1328,2023-08-21,15.083333
1329,2023-08-22,15.366667


In [44]:
pePM10 = pd.read_csv('../778project/data/airquality/auckland/PM10/penrosePM10_h.csv', skiprows=4)
pePM10

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,15.6
1,2020-01-01 01:00:00,21.1
2,2020-01-01 02:00:00,15.7
3,2020-01-01 03:00:00,13.8
4,2020-01-01 04:00:00,19.2
...,...,...
34669,2023-12-31 20:00:00,-0.3
34670,2023-12-31 21:00:00,2.4
34671,2023-12-31 22:00:00,0.0
34672,2023-12-31 23:00:00,2.0


In [45]:
pePM10.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'pm_2'}, inplace=True)
pePM10['Date'] = pd.to_datetime(pePM10['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
pePM10['Date'] = pd.to_datetime(pePM10['Date'])
pePM10 = pePM10.groupby('Date')['pm_2'].mean().reset_index()
pePM10 = pePM10[(pePM10['Date']>='2020-01-01') & (pePM10['Date']<='2023-08-23')]
pm10_df = pm10_df.merge(pePM10, on='Date', how='left')
pm10_df

,Date,pm_1,pm_2
0,2020-01-01,14.895833,12.666667
1,2020-01-02,8.429167,10.270833
2,2020-01-03,16.066667,19.512500
3,2020-01-04,21.095833,27.658333
4,2020-01-05,18.772727,21.782609
...,...,...,...
1326,2023-08-19,10.154167,13.970833
1327,2023-08-20,5.558333,6.634783
1328,2023-08-21,15.083333,18.812500
1329,2023-08-22,15.366667,21.466667


In [46]:
quPM10 = pd.read_csv('../778project/data/airquality/auckland/PM10/queenstreetPM10_h.csv', skiprows=4)
quPM10

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,18.0
1,2020-01-01 01:00:00,18.3
2,2020-01-01 02:00:00,16.8
3,2020-01-01 03:00:00,17.7
4,2020-01-01 04:00:00,19.6
...,...,...
29532,2023-08-23 00:00:00,32.2
29533,2023-08-23 01:00:00,28.6
29534,2023-08-23 02:00:00,28.7
29535,2023-08-23 03:00:00,25.7


In [47]:
quPM10.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'pm_3'}, inplace=True)
quPM10['Date'] = pd.to_datetime(quPM10['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
quPM10['Date'] = pd.to_datetime(quPM10['Date'])
quPM10 = quPM10.groupby('Date')['pm_3'].mean().reset_index()
quPM10 = quPM10[(quPM10['Date']>='2020-01-01') & (quPM10['Date']<='2023-08-23')]
pm10_df = pm10_df.merge(quPM10, on='Date', how='left')
pm10_df

,Date,pm_1,pm_2,pm_3
0,2020-01-01,14.895833,12.666667,14.587500
1,2020-01-02,8.429167,10.270833,10.870833
2,2020-01-03,16.066667,19.512500,19.975000
3,2020-01-04,21.095833,27.658333,25.450000
4,2020-01-05,18.772727,21.782609,20.147826
...,...,...,...,...
1326,2023-08-19,10.154167,13.970833,21.104167
1327,2023-08-20,5.558333,6.634783,12.345833
1328,2023-08-21,15.083333,18.812500,27.791667
1329,2023-08-22,15.366667,21.466667,27.691667


In [48]:
taPM10 = pd.read_csv('../778project/data/airquality/auckland/PM10/takapunaPM10_h.csv', skiprows=4)
taPM10

,Timestamp (UTC+12:00),Value (µg/m³)
0,2020-01-01 00:00:00,18.2
1,2020-01-01 01:00:00,22.0
2,2020-01-01 02:00:00,20.3
3,2020-01-01 03:00:00,18.4
4,2020-01-01 04:00:00,19.1
...,...,...
34124,2023-12-31 20:00:00,2.5
34125,2023-12-31 21:00:00,0.1
34126,2023-12-31 22:00:00,4.2
34127,2023-12-31 23:00:00,4.8


In [49]:
taPM10.rename(columns={'Timestamp (UTC+12:00)':'Date', 'Value (µg/m³)':'pm_4'}, inplace=True)
taPM10['Date'] = pd.to_datetime(taPM10['Date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
taPM10['Date'] = pd.to_datetime(taPM10['Date'])
taPM10 = taPM10.groupby('Date')['pm_4'].mean().reset_index()
taPM10 = taPM10[(taPM10['Date']>='2020-01-01') & (taPM10['Date']<='2023-08-23')]
pm10_df = pm10_df.merge(taPM10, on='Date', how='left')
pm10_df

,Date,pm_1,pm_2,pm_3,pm_4
0,2020-01-01,14.895833,12.666667,14.587500,12.216667
1,2020-01-02,8.429167,10.270833,10.870833,8.095833
2,2020-01-03,16.066667,19.512500,19.975000,15.787500
3,2020-01-04,21.095833,27.658333,25.450000,20.016667
4,2020-01-05,18.772727,21.782609,20.147826,17.252174
...,...,...,...,...,...
1326,2023-08-19,10.154167,13.970833,21.104167,12.275000
1327,2023-08-20,5.558333,6.634783,12.345833,5.350000
1328,2023-08-21,15.083333,18.812500,27.791667,14.566667
1329,2023-08-22,15.366667,21.466667,27.691667,15.579167


In [50]:
pm10_df['Daily Average PM10 (ug/m3)'] = pm10_df[['pm_1', 'pm_2', 'pm_3', 'pm_4']].mean(axis=1)
pm10_df = pm10_df[['Date', 'Daily Average PM10 (ug/m3)']]
pm10_df

,Date,Daily Average PM10 (ug/m3)
0,2020-01-01,13.591667
1,2020-01-02,9.416667
2,2020-01-03,17.835417
3,2020-01-04,23.555208
4,2020-01-05,19.488834
...,...,...
1326,2023-08-19,14.376042
1327,2023-08-20,7.472237
1328,2023-08-21,19.063542
1329,2023-08-22,20.026042


### Air quality

In [51]:
akl_df = full_range.merge(aqi_df, on='Date', how='left')
akl_df = akl_df.merge(no_df, on='Date', how='left')
akl_df = akl_df.merge(o3_df, on='Date', how='left')
akl_df = akl_df.merge(so2_df, on='Date', how='left')
akl_df = akl_df.merge(pm25_df, on='Date', how='left')
akl_df = akl_df.merge(pm10_df, on='Date', how='left')
akl_df

,Date,Daily Average AQI,Daily Average NOx (ug/m3),Daily Average O3 (ug/m3),Daily Average SO2 (ug/m3),Daily Average PM2.5 (ug/m3),Daily Average PM10 (ug/m3)
0,2020-01-01,36.125000,21.589583,29.608696,-0.004348,7.566667,13.591667
1,2020-01-02,20.895833,15.750000,37.869565,0.016667,5.120833,9.416667
2,2020-01-03,27.927083,27.144792,41.826087,0.004348,10.140278,17.835417
3,2020-01-04,43.343750,19.431522,38.608696,0.000000,9.586111,23.555208
4,2020-01-05,45.385417,32.413439,40.136364,-0.018182,8.267210,19.488834
...,...,...,...,...,...,...,...
1326,2023-08-19,36.270833,30.493750,48.765833,0.952174,8.277778,14.376042
1327,2023-08-20,20.770833,35.242391,52.997917,1.233333,4.033333,7.472237
1328,2023-08-21,28.436141,25.593750,62.969583,1.265217,7.970395,19.063542
1329,2023-08-22,41.171196,46.639130,50.296250,1.358333,8.440833,20.026042


## Weather

In [52]:
weather = pd.read_csv('../778project/data/weather/auckland_weather.csv', skiprows=10)
weather = weather[:-6]
weather

,Station,Day(Local_Date),WDir(Deg),WSpd(m/s),GustDir(Deg),GustSpd(m/s),WindRun(Km),Rain(mm),Tdry(C),TWet(C),...,Tgmin(C),ET05(C),ET10(C),ET20(C),ET30(C),ET100(C),Pmsl(hPa),Pstn(hPa),Sun(Hrs),Rad(MJ/m2)
0,41351,20180101.0,345,1.9,46,7.7,169,6.2,21.0,17.7,...,-,-,-,-,-,-,1010.4,1008.5,1.7,11.97
1,41351,20180102.0,49,2.2,9,8.2,277,0.0,21.0,19.7,...,-,-,-,-,-,-,1012.3,1010.4,12.4,31.39
2,41351,20180103.0,100,2.8,51,9.8,298,0.4,22.2,19.1,...,-,-,-,-,-,-,1011.4,1009.5,9.3,23.44
3,41351,20180104.0,52,3.8,67,18.5,614,43.2,21.4,20.4,...,-,-,-,-,-,-,1008.6,1006.7,0.0,4.93
4,41351,20180105.0,16,9.5,56,20.6,511,14.2,19.0,18.1,...,-,-,-,-,-,-,985.0,983.2,0.2,9.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,41351,20231227.0,237,1.1,230,8.2,182,0.0,21.8,19.2,...,14.3,-,21.2,21.1,-,18.5,1019.7,1017.8,9.7,23.95
2187,41351,20231228.0,37,6.0,39,13.4,440,3.6,21.1,19.1,...,10.3,-,20.6,21.2,-,18.9,1020.1,1018.2,7.0,21.08
2188,41351,20231229.0,20,2.8,40,13.4,284,52.8,21.8,21.1,...,17.9,-,20.6,20.7,-,18.8,1013.1,1011.2,0.0,6.17
2189,41351,20231230.0,306,2.5,309,9.3,229,0.0,21.8,21.3,...,19.1,-,20.9,20.9,-,19.4,1012.7,1010.8,8.9,22.58


In [53]:
weather['Day(Local_Date)'] = weather['Day(Local_Date)'].astype(str).str.split('.').str[0]
weather['Day(Local_Date)'] = pd.to_datetime(weather['Day(Local_Date)'], format='%Y%m%d')
weather = weather.rename(columns={'Day(Local_Date)':'Date'})
weather = weather[(weather['Date']>='2020-01-01') & (weather['Date']<='2023-08-23')]
weather = weather.drop(columns=['Station'])
weather

,Date,WDir(Deg),WSpd(m/s),GustDir(Deg),GustSpd(m/s),WindRun(Km),Rain(mm),Tdry(C),TWet(C),RH(%),...,Tgmin(C),ET05(C),ET10(C),ET20(C),ET30(C),ET100(C),Pmsl(hPa),Pstn(hPa),Sun(Hrs),Rad(MJ/m2)
730,2020-01-01,317,1.8,224,8.2,270,0.0,18.8,16.6,79.0,...,9.3,-,19.7,19.6,-,19.1,1018.6,1016.7,6.9,19.03
731,2020-01-02,226,3.5,196,11.8,324,0.0,17.8,15.6,78.0,...,13.3,-,19.2,19.6,-,19.1,1014.6,1012.7,9.4,21.61
732,2020-01-03,266,4.5,216,12.4,432,0.0,20.5,18.3,80.0,...,12.6,-,20.3,19.8,-,19.1,1008.0,1006.1,9.3,20.18
733,2020-01-04,222,6.5,215,15.4,462,0.0,17.8,13.2,56.0,...,13.8,-,18.4,19.2,-,19.2,1008.7,1006.8,7.2,18.62
734,2020-01-05,246,3.8,298,11.8,394,3.2,18.0,13.7,59.0,...,11.2,-,18.4,18.5,-,19.2,1012.5,1010.6,4.3,10.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056,2023-08-19,359,1.4,16,13.4,247,16.6,11.8,11.3,94.0,...,-0.4,-,9.0,9.6,-,12.4,1016.7,1014.8,0.6,4.41
2057,2023-08-20,349,1.5,304,9.3,288,5.0,13.1,12.9,97.0,...,10.7,-,11.1,10.7,-,12.2,1005.4,1003.5,0.8,5.14
2058,2023-08-21,250,7.0,237,15.4,337,4.8,10.8,9.5,84.0,...,7.1,-,10.5,10.8,-,12.2,1003.4,1001.5,3.4,7.08
2059,2023-08-22,189,2.2,177,11.3,211,0.0,12.6,9.9,69.0,...,0.6,-,9.8,10.3,-,12.1,1021.9,1019.9,10.3,14.64


## Traffic

In [54]:
tms = pd.read_csv('../778project/data/traffic/TMS_daily_traffic_counts_API.csv')
tms

,OBJECTID,startDate,siteID,regionName,SiteRef,classWeight,siteDescription,laneNumber,flowDirection,trafficCount
0,1,2017/12/31 12:00:00+00,39,05 - Gisborne,00200444,Light,200 m Nth of Bell Rd,2,2,2468.0
1,2,2017/12/31 12:00:00+00,39,05 - Gisborne,00200444,Heavy,200 m Nth of Bell Rd,2,2,73.0
2,3,2017/12/31 12:00:00+00,39,05 - Gisborne,00200444,Heavy,200 m Nth of Bell Rd,1,1,74.0
3,4,2017/12/31 12:00:00+00,39,05 - Gisborne,00200444,Light,200 m Nth of Bell Rd,1,1,3806.0
4,5,2017/12/31 12:00:00+00,40,05 - Gisborne,00200448,Heavy,Sth end of Whatatuna Bridge,1,1,58.5
...,...,...,...,...,...,...,...,...,...,...
4983425,4995973,2023/12/30 00:00:00+00,3223,02 - Auckland,01N19424,Light,Auckland Harbour Bridge - Classifier Site No 1...,1,1,8.0
4983426,4995974,2023/12/18 00:00:00+00,3545,02 - Auckland,01600040,Heavy,SH16 Sth of Hutchison Rd,1,1,282.5
4983427,4995975,2024/01/10 00:00:00+00,478627,02 - Auckland,01N40380,Light,SH1 Puhoi Off Ramp NB,1,6,1357.5
4983428,4995976,2023/12/24 00:00:00+00,3223,02 - Auckland,01N19424,Heavy,Auckland Harbour Bridge - Classifier Site No 1...,5,1,0.0


In [55]:
tms = tms[tms['regionName']=='02 - Auckland']
tms

,OBJECTID,startDate,siteID,regionName,SiteRef,classWeight,siteDescription,laneNumber,flowDirection,trafficCount
204,205,2017/12/31 12:00:00+00,334,02 - Auckland,01600024,Heavy,SH16 Nth of Coatesville Riverhead Highway,1,1,144.5
205,206,2017/12/31 12:00:00+00,334,02 - Auckland,01600024,Heavy,SH16 Nth of Coatesville Riverhead Highway,2,2,179.0
206,207,2017/12/31 12:00:00+00,334,02 - Auckland,01600024,Light,SH16 Nth of Coatesville Riverhead Highway,1,1,8498.5
207,208,2017/12/31 12:00:00+00,334,02 - Auckland,01600024,Light,SH16 Nth of Coatesville Riverhead Highway,2,2,10280.0
208,209,2017/12/31 12:00:00+00,340,02 - Auckland,01600058,Light,SH16 Nth of Kahikatea Flat Rd (Kaukapakapa Bri...,1,1,1701.5
...,...,...,...,...,...,...,...,...,...,...
4983425,4995973,2023/12/30 00:00:00+00,3223,02 - Auckland,01N19424,Light,Auckland Harbour Bridge - Classifier Site No 1...,1,1,8.0
4983426,4995974,2023/12/18 00:00:00+00,3545,02 - Auckland,01600040,Heavy,SH16 Sth of Hutchison Rd,1,1,282.5
4983427,4995975,2024/01/10 00:00:00+00,478627,02 - Auckland,01N40380,Light,SH1 Puhoi Off Ramp NB,1,6,1357.5
4983428,4995976,2023/12/24 00:00:00+00,3223,02 - Auckland,01N19424,Heavy,Auckland Harbour Bridge - Classifier Site No 1...,5,1,0.0


In [56]:
tms['startDate'] = pd.to_datetime(tms['startDate'], dayfirst=True)
tms['startDate'] = tms['startDate'].dt.date
tms['startDate'] = pd.to_datetime(tms['startDate'])
tms.dtypes

C:\Users\Alvin Zhao\AppData\Local\Temp\ipykernel_8128\2149695656.py:1: UserWarning: Parsing dates in %Y/%m/%d %H:%M:%S%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  tms['startDate'] = pd.to_datetime(tms['startDate'], dayfirst=True)
C:\Users\Alvin Zhao\AppData\Local\Temp\ipykernel_8128\2149695656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tms['startDate'] = pd.to_datetime(tms['startDate'], dayfirst=True)
C:\Users\Alvin Zhao\AppData\Local\Temp\ipykernel_8128\2149695656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

OBJECTID                    int64
startDate          datetime64[ns]
siteID                      int64
regionName                 object
SiteRef                    object
classWeight                object
siteDescription            object
laneNumber                  int64
flowDirection               int64
trafficCount              float64
dtype: object

In [57]:
tms_light = tms[tms['classWeight']=='Light']
tms_light = tms_light[(tms_light['startDate']>='2020-01-01') & (tms_light['startDate']<='2023-08-23')]
tms_light

,OBJECTID,startDate,siteID,regionName,SiteRef,classWeight,siteDescription,laneNumber,flowDirection,trafficCount
1560883,1563803,2020-01-01,334,02 - Auckland,01600024,Light,SH16 Nth of Coatesville Riverhead Highway,2,2,13013.0
1560885,1563805,2020-01-01,334,02 - Auckland,01600024,Light,SH16 Nth of Coatesville Riverhead Highway,1,1,11153.5
1560887,1563807,2020-01-01,340,02 - Auckland,01600058,Light,SH16 Nth of Kahikatea Flat Rd (Kaukapakapa Bri...,1,1,3106.5
1560888,1563808,2020-01-01,340,02 - Auckland,01600058,Light,SH16 Nth of Kahikatea Flat Rd (Kaukapakapa Bri...,2,2,4402.0
1560910,1563830,2020-01-01,389,02 - Auckland,01N00383,Light,Sth of Puhoi Rd,1,1,16724.5
...,...,...,...,...,...,...,...,...,...,...
4928275,4940823,2021-05-11,1142,02 - Auckland,01N20440,Light,PANAMA RD - Telemetry Site 79 - NB,3,2,20636.5
4928278,4940826,2023-04-06,2286,02 - Auckland,02030011,Light,SH20 Coronation Rd On Ramp NB,1,3,9314.0
4928287,4940835,2022-05-12,2225,02 - Auckland,01640006,Light,SH16 St Lukes Rd Off Ramp EB,1,6,11071.0
4928294,4940842,2022-05-06,2042,02 - Auckland,01N60426,Light,SH1 Fanshawe St Off Ramp SB,2,4,4228.0


In [58]:
tms_light = tms_light.groupby('startDate')['trafficCount'].mean().reset_index()
tms_light

,startDate,trafficCount
0,2020-01-01,8631.990268
1,2020-01-02,10138.637470
2,2020-01-03,9291.510949
3,2020-01-04,8757.687348
4,2020-01-05,11109.727941
...,...,...
1326,2023-08-19,10735.478261
1327,2023-08-20,8876.103198
1328,2023-08-21,10663.613043
1329,2023-08-22,11362.219718


In [59]:
tms_heavy = tms[tms['classWeight']=='Heavy']
tms_heavy = tms_heavy[(tms_heavy['startDate']>='2020-01-01') & (tms_heavy['startDate']<='2023-08-23')]
tms_heavy = tms_heavy.groupby('startDate')['trafficCount'].mean().reset_index()
tms_heavy

,startDate,trafficCount
0,2020-01-01,358.250000
1,2020-01-02,651.330357
2,2020-01-03,447.879464
3,2020-01-04,352.015625
4,2020-01-05,888.281250
...,...,...
1326,2023-08-19,434.606557
1327,2023-08-20,271.606996
1328,2023-08-21,814.182377
1329,2023-08-22,867.473896


In [60]:
tms_light = tms_light.rename(columns={'startDate':'Date', 'trafficCount':'lightCount'})
tms_heavy = tms_heavy.rename(columns={'startDate':'Date', 'trafficCount':'heavyCount'})
tms_light

,Date,lightCount
0,2020-01-01,8631.990268
1,2020-01-02,10138.637470
2,2020-01-03,9291.510949
3,2020-01-04,8757.687348
4,2020-01-05,11109.727941
...,...,...
1326,2023-08-19,10735.478261
1327,2023-08-20,8876.103198
1328,2023-08-21,10663.613043
1329,2023-08-22,11362.219718


In [61]:
tms_heavy

,Date,heavyCount
0,2020-01-01,358.250000
1,2020-01-02,651.330357
2,2020-01-03,447.879464
3,2020-01-04,352.015625
4,2020-01-05,888.281250
...,...,...
1326,2023-08-19,434.606557
1327,2023-08-20,271.606996
1328,2023-08-21,814.182377
1329,2023-08-22,867.473896


## Merged

In [62]:
akl_df = akl_df.merge(weather, on='Date', how='left')
akl_df

,Date,Daily Average AQI,Daily Average NOx (ug/m3),Daily Average O3 (ug/m3),Daily Average SO2 (ug/m3),Daily Average PM2.5 (ug/m3),Daily Average PM10 (ug/m3),WDir(Deg),WSpd(m/s),GustDir(Deg),...,Tgmin(C),ET05(C),ET10(C),ET20(C),ET30(C),ET100(C),Pmsl(hPa),Pstn(hPa),Sun(Hrs),Rad(MJ/m2)
0,2020-01-01,36.125000,21.589583,29.608696,-0.004348,7.566667,13.591667,317,1.8,224,...,9.3,-,19.7,19.6,-,19.1,1018.6,1016.7,6.9,19.03
1,2020-01-02,20.895833,15.750000,37.869565,0.016667,5.120833,9.416667,226,3.5,196,...,13.3,-,19.2,19.6,-,19.1,1014.6,1012.7,9.4,21.61
2,2020-01-03,27.927083,27.144792,41.826087,0.004348,10.140278,17.835417,266,4.5,216,...,12.6,-,20.3,19.8,-,19.1,1008.0,1006.1,9.3,20.18
3,2020-01-04,43.343750,19.431522,38.608696,0.000000,9.586111,23.555208,222,6.5,215,...,13.8,-,18.4,19.2,-,19.2,1008.7,1006.8,7.2,18.62
4,2020-01-05,45.385417,32.413439,40.136364,-0.018182,8.267210,19.488834,246,3.8,298,...,11.2,-,18.4,18.5,-,19.2,1012.5,1010.6,4.3,10.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,2023-08-19,36.270833,30.493750,48.765833,0.952174,8.277778,14.376042,359,1.4,16,...,-0.4,-,9.0,9.6,-,12.4,1016.7,1014.8,0.6,4.41
1327,2023-08-20,20.770833,35.242391,52.997917,1.233333,4.033333,7.472237,349,1.5,304,...,10.7,-,11.1,10.7,-,12.2,1005.4,1003.5,0.8,5.14
1328,2023-08-21,28.436141,25.593750,62.969583,1.265217,7.970395,19.063542,250,7.0,237,...,7.1,-,10.5,10.8,-,12.2,1003.4,1001.5,3.4,7.08
1329,2023-08-22,41.171196,46.639130,50.296250,1.358333,8.440833,20.026042,189,2.2,177,...,0.6,-,9.8,10.3,-,12.1,1021.9,1019.9,10.3,14.64


In [63]:
akl_df = akl_df.merge(tms_light, on='Date')
akl_df = akl_df.merge(tms_heavy, on='Date')
akl_df

,Date,Daily Average AQI,Daily Average NOx (ug/m3),Daily Average O3 (ug/m3),Daily Average SO2 (ug/m3),Daily Average PM2.5 (ug/m3),Daily Average PM10 (ug/m3),WDir(Deg),WSpd(m/s),GustDir(Deg),...,ET10(C),ET20(C),ET30(C),ET100(C),Pmsl(hPa),Pstn(hPa),Sun(Hrs),Rad(MJ/m2),lightCount,heavyCount
0,2020-01-01,36.125000,21.589583,29.608696,-0.004348,7.566667,13.591667,317,1.8,224,...,19.7,19.6,-,19.1,1018.6,1016.7,6.9,19.03,8631.990268,358.250000
1,2020-01-02,20.895833,15.750000,37.869565,0.016667,5.120833,9.416667,226,3.5,196,...,19.2,19.6,-,19.1,1014.6,1012.7,9.4,21.61,10138.637470,651.330357
2,2020-01-03,27.927083,27.144792,41.826087,0.004348,10.140278,17.835417,266,4.5,216,...,20.3,19.8,-,19.1,1008.0,1006.1,9.3,20.18,9291.510949,447.879464
3,2020-01-04,43.343750,19.431522,38.608696,0.000000,9.586111,23.555208,222,6.5,215,...,18.4,19.2,-,19.2,1008.7,1006.8,7.2,18.62,8757.687348,352.015625
4,2020-01-05,45.385417,32.413439,40.136364,-0.018182,8.267210,19.488834,246,3.8,298,...,18.4,18.5,-,19.2,1012.5,1010.6,4.3,10.36,11109.727941,888.281250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,2023-08-19,36.270833,30.493750,48.765833,0.952174,8.277778,14.376042,359,1.4,16,...,9.0,9.6,-,12.4,1016.7,1014.8,0.6,4.41,10735.478261,434.606557
1327,2023-08-20,20.770833,35.242391,52.997917,1.233333,4.033333,7.472237,349,1.5,304,...,11.1,10.7,-,12.2,1005.4,1003.5,0.8,5.14,8876.103198,271.606996
1328,2023-08-21,28.436141,25.593750,62.969583,1.265217,7.970395,19.063542,250,7.0,237,...,10.5,10.8,-,12.2,1003.4,1001.5,3.4,7.08,10663.613043,814.182377
1329,2023-08-22,41.171196,46.639130,50.296250,1.358333,8.440833,20.026042,189,2.2,177,...,9.8,10.3,-,12.1,1021.9,1019.9,10.3,14.64,11362.219718,867.473896


In [64]:
akl_df.columns

Index(['Date', 'Daily Average AQI', 'Daily Average NOx (ug/m3)',
       'Daily Average O3 (ug/m3)', 'Daily Average SO2 (ug/m3)',
       'Daily Average PM2.5 (ug/m3)', 'Daily Average PM10 (ug/m3)',
       'WDir(Deg)', 'WSpd(m/s)', 'GustDir(Deg)', 'GustSpd(m/s)', 'WindRun(Km)',
       'Rain(mm)', 'Tdry(C)', 'TWet(C)', 'RH(%)', 'Tmax(C)', 'Tmin(C)',
       'Tgmin(C)', 'ET05(C)', 'ET10(C)', 'ET20(C)', 'ET30(C)', 'ET100(C)',
       'Pmsl(hPa)', 'Pstn(hPa)', 'Sun(Hrs)', 'Rad(MJ/m2)', 'lightCount',
       'heavyCount'],
      dtype='object')

In [70]:
akl_df.to_csv('auckland_data.csv', index=False)